In [9]:
import sklearn as sk
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import os


In [5]:
num_clusts = range(40)


In [13]:
pd.read_csv(os.getcwd() + '/feat_extraction/movies_with_feats.csv')
# I think pedro means STRING here. 
        # I will now try to segment the entire dialogue into SENTENCES!!!! 
        # SO that THIS way, we can also attempt to get % of senteces with positive/negative polarity
        sentence_tokenizer.tokenize

,Unnamed: 0,Processed Title,Success,n_unique_words_char_1,n_unique_words_char_2,n_unique_words_char_3,n_unique_words_char_4,n_unique_words_char_5,FK_read_level_char_1,FK_read_level_char_2,...,n_curse_words_char_2,n_curse_words_char_3,n_curse_words_char_4,n_curse_words_char_5,n_mentions_others_char_1,n_mentions_others_char_2,n_mentions_others_char_3,n_mentions_others_char_4,n_mentions_others_char_5,main_char_rel_diag_length
0,Avatar,Avatar,1,670,565,252,425,276,2,2,...,9,1,5,6,35,26,10,2,1,39.537232
1,The Dark Knight Rises,"Dark-Knight-Rises,-The",1,531,514,505,417,434,2,2,...,2,0,0,0,34,36,5,26,19,23.705825
2,The Avengers,"Avengers,-The",1,560,622,359,425,205,2,3,...,0,1,0,0,36,20,18,11,2,30.146047
3,Pirates of the Caribbean: Dead Man's Chest,Pirates-of-the-Caribbean-Dead-Man's-Chest,1,629,373,285,379,281,2,1,...,0,0,0,0,20,8,2,1,2,36.743621
4,Men in Black 3,Men-in-Black-3,1,1206,867,234,209,185,2,3,...,4,0,0,2,180,88,17,16,11,57.328163
5,King Kong,King-Kong,1,637,260,231,245,231,2,1,...,0,0,0,0,91,34,18,15,21,48.472248
6,Guardians of the Galaxy Vol 2,Guardians-of-the-Galaxy-Vol-2,1,589,463,348,480,319,2,2,...,6,0,3,6,33,32,21,7,10,29.971018
7,Cars 2,Cars-2,1,688,389,410,376,232,2,1,...,0,1,0,0,42,76,43,27,6,41.850962
8,2012,2012,1,366,486,217,297,206,1,3,...,1,0,3,0,28,12,22,2,8,26.847535
9,Terminator Salvation,Terminator-Salvation,1,447,357,281,224,211,1,2,...,6,0,0,0,5,16,12,2,2,37.711335


In [14]:
#so, we will actually have to break these guys down into senteces... 
# so I will have to do the sentence tokenizer :) 
def jacardian_distance(document_1_data, document_2_data):

    
    #for key in processed_article_hash.keys(): 
    #art_uniq_ws[key] = []
    #for sent in processed_article_hash[key]:
    #    art_uniq_ws[key].extend(sent)
    #    art_uniq_ws[key] = list(set(art_uniq_ws[key]))
    d1_uniqs = []
    for sent in document_1_data:
        d1_uniqs.extend(sent)
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    for sent in document_2_data:
        d2_uniqs.extend(sent)
        d2_uniqs = list(set(d2_uniqs))
        
    words_in_doc_1_not_in_doc_2 = [value for value in d1_uniqs if value not in d2_uniqs]
    words_in_doc_2_not_in_doc_1 = [value for value in d2_uniqs if value not in d1_uniqs]
    words_in_both_doc_1_and_doc_2 = [value for value in d1_uniqs if value in d2_uniqs]
    
    
    
    #overlap = [value for value in d1_uniqs if value in d2_uniqs] 
    #in1Not2 = [value for value in d1_uniqs if value not in d2_uniqs]
    #in2Not1 = [value for value in d2_uniqs if value not in d1_uniqs]
   # print(len(in1Not2))
    #print(len(in2Not1))
    #print(len(sv))
   # print(len(sv)+len(in2Not1)+len(in1Not2))

   # sv = d2_uniqs + d1_uniqs
   # sv = list(set(sv))
   # print(len(overlap))
    
    
    
    jacardian = len(words_in_both_doc_1_and_doc_2)/(len(words_in_both_doc_1_and_doc_2) + len(words_in_doc_1_not_in_doc_2) + len(words_in_doc_2_not_in_doc_1)) # divide the counts appropiately
    
    return jacardian
    
    
    
#Cosine Similarity: Here we want to create vector representations for each document. Specifically, we want to come up with a vector that is based on the list of all words that occur across both documents. Then for each document we will create a vector that includes the counts of the number of time a word occurs in the document.

#So if the document 1 is: "the ship sails at midnight" and document 2 is: "the crow flies at noon." We would be creating a vector like: [the, ship, sails, at, midnight, crow, flies, noon]. Then we would calculate the values of the vector for each document. For document 1: [1,1,1,1,1,0,0,0] and for document 2: [1,0,0,1,0,1,1,1]. With these two vectors we would simply take the dot product and that would provide the cosine similarity. 

def cosine_similarity(document_1_data, document_2_data):
    document_vector_word_index = [] # here fill this with an ordered list of all the unique words across both documents
    d1_uniqs = []
    d1_nun = []
    for sent in document_1_data:
        d1_uniqs.extend(sent)
        d1_nun.extend(sent)
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    d2_nun = []
    for sent in document_2_data:
        d2_uniqs.extend(sent)
        d2_uniqs = list(set(d2_uniqs))
        d2_nun.extend(sent)

    
    all_words = d1_nun+d2_nun   

    overlap = d1_uniqs + d2_uniqs 
    # so this is all of the unique words across the two documents
    document_vector_word_index += list(set(overlap))
    

    document_1_vector = list(np.zeros(len(document_vector_word_index)))  # fill in the array with the frequency of the words in the document
    document_2_vector = list(np.zeros(len(document_vector_word_index))) # fill in the array with the frequency of the words in the document

    for wOrd in d1_nun:
        document_1_vector[document_vector_word_index.index(wOrd)] +=1
    
    for wOrd in d2_nun:
        document_2_vector[document_vector_word_index.index(wOrd)] +=1
    
    d1_norm = (np.sum(np.array(document_1_vector)**2))**(1/2)
    d2_norm = (np.sum(np.array(document_2_vector)**2))**(1/2)
    
    cosSim = np.dot(document_1_vector,document_2_vector)/(d1_norm*d2_norm)


    
    return (cosSim)
#document_1_vector,document_2_vector,document_vector_word_index, are the things I tried to return in testing
#dot_product_of_two_document_vectors # you can refer to the numpy information on how to calculate the dot product of vectors

In [ ]:
# create a variable to store your table data... you could use a hash or some other data structure. 
# We just want it to identify which document is being compared to which other document.

data_structure_for_jacard_similarity = []
data_structure_for_cosine_similarity = []
# so I think what I'm ACTUALLY going to do here 
# is just append a new column
# and then say the column is .... 
# so we are comparing each character to every OTHER character (including themselves)
for char_1_index in char_series.index:
    for char_2_index in char_series.index:
        # we have the nested for loops as one way to compare each document to each other document
        data_structure_for_jacard_similarity.append(jacardian_distance(char_series[char_1_index], char_series[char_2_index]))
        data_structure_for_cosine_similarity.append(cosine_similarity(char_series[char_1_index], char_series[char_2_index]))

# finally, find some way to present this data back. Either as a straight table or a heatmap.
# so... yeah.... I can probably keep the rows the same as what they were... TECHNICALLY.... 
colz = char_series.index
rowz = char_series.index


for movChar in char_series_index:
    

In [15]:
pd.Series(np.nan,index = char_series.index)

NameError: name 'char_series' is not defined